In [2]:
from google.colab import drive
drive.mount('./gdrive', force_remount=True)
root_dir = "./gdrive/My Drive/Colab Notebooks/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at ./gdrive


In [0]:
import os
os.listdir("./gdrive/My Drive/CUB_200_2011")
input_dir = "./gdrive/My Drive/CUB_200_2011"

In [0]:
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import pickle
import random


class Img_embed(nn.Module):
    def __init__(self, input_size, hidden_dim, output_size):
        super(Img_embed, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_dim)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(hidden_dim, output_size)
  
    def forward(self,x):
        x = x.view(1, -1)
        output = self.fc1(x)
        output = self.relu(output)
        output = F.normalize(self.fc2(output), dim=0, p=2)

        return output

class Text_embed(nn.Module):
    def __init__(self, input_size, hidden_dim, output_size):
        super(Text_embed, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_dim)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(hidden_dim, output_size)

    def forward(self,x):
        x = x.view(1, -1)
        output = self.fc1(x)
        output = self.relu(output)
        output = F.normalize(self.fc2(output), dim=0, p=2)

        return output



In [0]:
with open (os.path.join(input_dir,"img.pkl"),'rb') as f:
    imgs = pickle.load(f)

# with open ("txts.pkl",'rb') as f:
#     txts = pickle.load(f)
with open (os.path.join(input_dir,"encoded_txts.pkl"),'rb') as f:
    encoded_txts = pickle.load(f)


for i in range(len(encoded_txts)):
    encoded_txts[i] = torch.tensor(encoded_txts[i])

pairs = list(zip(imgs, encoded_txts))

random.shuffle(pairs)

train_idx = int(len(pairs) * 0.70)

train = pairs[:train_idx]
test = pairs[train_idx:]



In [0]:
test_idx = np.random.randint(len(test),size=10)

In [0]:
def save_model(epoch):
  pt = os.path.join(input_dir,'img_model_best.pth')
  checkpoint = {'start_epoch':epoch,'best_loss':best_loss,'img_model':img_model,'txt_model':txt_model,}
  torch.save(checkpoint,pt)
  print('checkpoint saved at {}'.format(pt))

In [0]:
epoch_num = 20


import time
start = time.time()

img_model = Img_embed(256**2, 64**2, 300)
txt_model = Text_embed(300, 300, 300)

print(os.path.exists(os.path.join(input_dir,'img_model_best.pth')))
# define models
if os.path.exists(os.path.join(input_dir,'img_model_best.pth')):

  pt = os.path.join(input_dir,'img_model_best.pth')
  checkpoint = torch.load(pt)
  img_model = checkpoint['img_model']
  txt_model = checkpoint['txt_model']
  best_loss = checkpoint['best_loss']
  start_epoch = checkpoint['start_epoch']

else:
  best_loss = float('inf')
  start_epoch = 0

losses_img = []
losses_cap = []
cnt = 0 

triplet_loss_img = nn.TripletMarginLoss(margin=0.01, p=2)
triplet_loss_cap = nn.TripletMarginLoss(margin=0.01, p=2)
optm_img = optim.SGD(img_model.parameters(), lr=0.001, momentum=0.9)
optm_txt = optim.SGD(txt_model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(start_epoch,epoch_num):
    random.shuffle(train)
    for j in range(0, len(train)-1, 2):
        optm_img.zero_grad()
        optm_txt.zero_grad()
        
        anchor_img = train[j][0]
        anchor_cap = train[j][1]
        neg_img = train[j+1][0]
        neg_cap = train[j+1][1]
        
        # forward
        anchor_img = img_model(anchor_img)
        neg_img = img_model(neg_img)
        anchor_cap = txt_model(anchor_cap)
        neg_cap = txt_model(neg_cap)
        
        # loss 
        loss_img = triplet_loss_img(anchor_img, anchor_cap, neg_cap) + triplet_loss_img(neg_img, neg_cap,anchor_cap)
        loss_cap = triplet_loss_cap(neg_cap, neg_img, anchor_img) + triplet_loss_cap(anchor_cap,anchor_img, neg_img)
        
        losses_img.append(loss_img)
        losses_cap.append(loss_cap)
        cnt += 1 
        
        # backward
        loss_img.backward( retain_graph=True)
        loss_cap.backward( retain_graph=True)
        
        # optim step
        optm_img.step()
        optm_txt.step()


        if j % 100 == 0:
            print("epoch{}/{} --- Avg train Img Loss: {:.3f}, Avg train Text Loss: {:.3f}, time:{:.3f} per 100 samples"
            .format(epoch+1, epoch_num, sum(losses_img)/cnt, sum(losses_cap)/cnt,time.time()-start))
            start = time.time()
            
            with torch.no_grad():
              img_model.eval()
              txt_model.eval()
              test_img_loss = []
              test_cap_loss = []
              # forward
              for k in test_idx:
                anchor_img = test[k][0]
                anchor_cap = test[k][1]
                neg_img = test[k+1][0]
                neg_cap = test[k+1][1]
          
                anchor_img = img_model(anchor_img)
                neg_img = img_model(neg_img)
                anchor_cap = txt_model(anchor_cap)
                neg_cap = txt_model(neg_cap)
                
                # loss 
                loss_img = triplet_loss_img(anchor_img, anchor_cap, neg_cap) + triplet_loss_img(neg_img, neg_cap,anchor_cap)
                loss_cap = triplet_loss_cap(neg_cap, neg_img, anchor_img) + triplet_loss_cap(anchor_cap,anchor_img, neg_img)

                test_img_loss.append(loss_img)
                test_cap_loss.append(loss_cap)

              cur_loss_total = (sum(test_cap_loss) + sum(test_cap_loss))/len(test_idx)
              if cur_loss_total < best_loss:
                save_model(epoch)
                best_loss = cur_loss_total

              print("epoch{}/{} --- Avg test Img Loss: {:.3f}, Avg test Text Loss: {:.3f}"
              .format(epoch+1, epoch_num, sum(test_img_loss)/len(test_img_loss), sum(test_cap_loss)/len(test_cap_loss)))

              img_model.train()
              txt_model.train()

True
epoch3/20 --- Avg train Img Loss: 0.000, Avg train Text Loss: 0.336, time:4.896 per 100 samples


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Img_embed. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Text_embed. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


checkpoint saved at ./gdrive/My Drive/CUB_200_2011/img_model_best.pth
epoch3/20 --- Avg test Img Loss: 0.189, Avg test Text Loss: 0.355
epoch3/20 --- Avg train Img Loss: 0.239, Avg train Text Loss: 0.603, time:35.756 per 100 samples
epoch3/20 --- Avg test Img Loss: 0.189, Avg test Text Loss: 0.364
epoch3/20 --- Avg train Img Loss: 0.217, Avg train Text Loss: 0.642, time:30.399 per 100 samples
epoch3/20 --- Avg test Img Loss: 0.189, Avg test Text Loss: 0.364
epoch3/20 --- Avg train Img Loss: 0.217, Avg train Text Loss: 0.637, time:30.325 per 100 samples
epoch3/20 --- Avg test Img Loss: 0.189, Avg test Text Loss: 0.355
epoch3/20 --- Avg train Img Loss: 0.211, Avg train Text Loss: 0.634, time:30.118 per 100 samples
epoch3/20 --- Avg test Img Loss: 0.189, Avg test Text Loss: 0.355
epoch3/20 --- Avg train Img Loss: 0.216, Avg train Text Loss: 0.652, time:30.502 per 100 samples
epoch3/20 --- Avg test Img Loss: 0.189, Avg test Text Loss: 0.364
epoch3/20 --- Avg train Img Loss: 0.214, Avg trai

In [0]:
if os.path.exists(os.path.join(input_dir,'img_model_best.pth')):

  pt = os.path.join(input_dir,'img_model_best.pth')
  checkpoint = torch.load(pt)
  img_model = checkpoint['img_model']
  txt_model = checkpoint['txt_model']
  best_loss = checkpoint['best_loss']
  start_epoch = checkpoint['start_epoch']